In [35]:
#!pip install ortools

In [36]:
from __future__ import print_function
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [37]:
#site 0 data
scenario1 = pd.read_csv("scenario1_0.csv")

In [38]:
scenario1.head()

,population,required_gdus,scenario_1_harvest_quantity,week_original_plant_date
0,p_56,1259,270,1
1,p_57,1259,389,1
2,p_58,1259,249,1
3,p_59,1356,100,1
4,p_60,1356,296,1


In [39]:
#sort scenario
scenario1.sort_values(by=['population'], inplace=True)

In [40]:
values = [] #
weights = [] #scenario 1 weight using GDU
week = []

In [41]:
for index, row in scenario1.iterrows():
    week.append(row["week_original_plant_date"])

In [42]:
week = list(set(week))

In [43]:
#filter scenario for 1 week
scenario = scenario1[scenario1['week_original_plant_date'] == 1]

In [44]:
for index, row in scenario.iterrows():
    weights.append(row["required_gdus"])
    values.append(row["scenario_1_harvest_quantity"])

In [54]:
scenario

,population,required_gdus,scenario_1_harvest_quantity,week_original_plant_date
0,p_56,1259,270,1
1,p_57,1259,389,1
2,p_58,1259,249,1
3,p_59,1356,100,1
4,p_60,1356,296,1
5,p_61,1259,104,1
6,p_62,1356,267,1
7,p_63,1356,268,1
8,p_64,1356,174,1
9,p_65,1259,76,1


In [62]:
data = {}
data['weights'] = values#weights
data['values'] = weights
data['items'] = list(range(len(weights)))
data['num_items'] = len(weights)

In [63]:
num_bins = 1#len(week)
data['bins'] = list(range(num_bins))
capacities = [7000 for i in range(0,num_bins)]
data['bin_capacities'] = capacities

In [64]:
# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

In [65]:
# Constraints
# Each item can be in at most one bin.
for i in data['items']:
    solver.Add(sum(x[i, j] for j in data['bins']) <= 1)


In [66]:
# The amount packed in each bin cannot exceed its capacity.
for j in data['bins']:
    solver.Add(
        sum(x[(i, j)] * data['weights'][i]
            for i in data['items']) <= data['bin_capacities'][j])

In [67]:
# Objective
objective = solver.Objective()

for i in data['items']:
    for j in data['bins']:
        objective.SetCoefficient(x[(i, j)], data['values'][i])
objective.SetMaximization()

In [68]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Total packed value:', objective.Value())
    total_weight = 0
    for j in data['bins']:
        bin_weight = 0
        bin_value = 0
        print('Bin ', j, '\n')
        for i in data['items']:
            if x[i, j].solution_value() > 0:
                print('Item', i, '- weight:', data['weights'][i], ' value:',
                      data['values'][i])
                bin_weight += data['weights'][i]
                bin_value += data['values'][i]
        print('Packed bin weight:', bin_weight)
        print('Packed bin value:', bin_value)
        print()
        total_weight += bin_weight
    print('Total packed weight:', total_weight)
else:
    print('The problem does not have an optimal solution.')

Total packed value: 25749.0
Bin  0 

Item 0 - weight: 270  value: 1259
Item 1 - weight: 389  value: 1259
Item 2 - weight: 249  value: 1259
Item 3 - weight: 100  value: 1356
Item 4 - weight: 296  value: 1356
Item 5 - weight: 104  value: 1259
Item 6 - weight: 267  value: 1356
Item 7 - weight: 268  value: 1356
Item 8 - weight: 174  value: 1356
Item 9 - weight: 76  value: 1259
Item 10 - weight: 161  value: 1259
Item 11 - weight: 151  value: 1259
Item 12 - weight: 177  value: 1259
Item 13 - weight: 174  value: 1356
Item 14 - weight: 354  value: 1259
Item 15 - weight: 236  value: 1259
Packed bin weight: 3446
Packed bin value: 20726

Total packed weight: 3446
